In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import skorch
import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from skorch.helper import predefined_split

In [2]:
class MyClassifier(nn.Module):
    def __init__(self):
        super(MyClassifier, self).__init__()
        self.model = models.efficientnet_b0(pretrained=True)
        
        for param in self.model.parameters():
            param.requires_grad = False
        
        self.fc1 = nn.Sequential(
            nn.Linear(1000, 64),
            nn.ReLU()
        )
        self.fc2 = nn.Sequential(
            nn.Dropout(0.25),
            nn.Linear(64, 10)
        )
    
    def embed(self, x):
        return self.fc1(self.model(x))
    
    def forward(self, x):
        resnet_out = self.embed(x)
        return F.softmax(self.fc2(resnet_out), dim=-1)

In [4]:
from torchinfo import summary

summary(MyClassifier(), input_size=(1, 3, 96, 96))

Layer (type:depth-idx)                                       Output Shape              Param #
MyClassifier                                                 --                        --
├─EfficientNet: 1-1                                          [1, 1000]                 --
│    └─Sequential: 2-1                                       [1, 1280, 3, 3]           --
│    │    └─ConvNormActivation: 3-1                          [1, 32, 48, 48]           (928)
│    │    └─Sequential: 3-2                                  [1, 16, 48, 48]           (1,448)
│    │    └─Sequential: 3-3                                  [1, 24, 24, 24]           (16,714)
│    │    └─Sequential: 3-4                                  [1, 40, 12, 12]           (46,640)
│    │    └─Sequential: 3-5                                  [1, 80, 6, 6]             (242,930)
│    │    └─Sequential: 3-6                                  [1, 112, 6, 6]            (543,148)
│    │    └─Sequential: 3-7                                  

In [3]:
data = datasets.STL10(root='stl10', download=True, transform=transforms.ToTensor())

Files already downloaded and verified


In [4]:
net = skorch.NeuralNetClassifier(
    MyClassifier,
    max_epochs=20,
    train_split=predefined_split(data),
    lr=0.1,
    batch_size=128,
    device='cuda',
    # criterion=
)

In [6]:
trained = net.fit(data, y=None)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        1.9202       0.6558        1.0996  5.5114
      2        1.0588       0.7628        0.7346  4.4213
      3        0.9112       0.7814        0.6694  4.4353
      4        0.8333       0.8110        0.5697  4.4731
      5        0.7806       0.8290        0.5171  4.3940
      6        0.7350       0.8452        0.4797  4.5165
      7        0.7150       0.8536        0.4498  4.7205
      8        0.6882       0.8662        0.4207  4.6702
      9        0.6508       0.8754        0.4006  4.5942
     10        0.6382       0.8832        0.3647  4.7451
     11        0.6272       0.8916        0.3447  4.5875
     12        0.5993       0.8970        0.3312  4.5117
     13        0.5757       0.8994        0.3190  4.6123
     14        0.5846       0.8992        0.3163  4.6317
     15        0.5586       0.9128        0.2905  4.6371
     16        0.5380       0.9

In [ ]:
from random import randint
xs, ys = zip(*[data[randint(0, len(data))] for _ in range(500)])
xs = torch.stack(xs)

In [ ]:
from tensorboardX import SummaryWriter

with SummaryWriter() as writer:
    writer.add_embedding(trained.module_.embed(xs.to('cuda')),
                         metadata=ys,
                         label_img=xs)